In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
train = pd.read_csv('data/dm3/open100.csv')

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(sorted(train.data_ind.unique()))
print(train.data_ind.unique().shape)

In [ ]:
train.loc[train.data_ind.isin(['15'])].loc[train.signal.isin(['0'])].Z.unique()

In [ ]:
sss = sorted(train.Z.unique())
min(np.array(sss[1:]) - np.array(sss[:-1]))

In [ ]:
train.loc[train.data_ind.isin(['54'])].hist(column='X', by='signal', bins=59)
train.loc[train.data_ind.isin(['54'])].hist(column='X', bins=59)
train.loc[train.data_ind.isin(['54'])].hist(column='Y', by='signal', bins=59)
train.loc[train.data_ind.isin(['54'])].hist(column='Y', bins=59)
train.loc[train.data_ind.isin(['54'])].hist(column='Z', by='signal', bins=58)
train.loc[train.data_ind.isin(['54'])].hist(column='Z', bins=58)
plt.show()

In [ ]:
test = pd.read_csv('data/dm3/test_50.csv')

In [ ]:
print(sorted(test.data_ind.unique()))

In [ ]:
test.loc[test.data_ind.isin(['2725'])].hist(column='X', bins=100)
test.loc[test.data_ind.isin(['2725'])].hist(column='Y', bins=100)
test.loc[test.data_ind.isin(['2725'])].hist(column='Z', bins=100)
plt.show()

In [ ]:
test.loc[test.data_ind.isin(['2705'])].Z.unique()

In [ ]:
def find_origin(array, index):
    raw_array = array.loc[array.data_ind.isin([index])].Z.unique()
    value = list(filter(lambda x: np.trunc(x) != x, raw_array))[0]
    subarray = array.loc[array.data_ind.isin([index])]
    return subarray.loc[subarray.Z.isin([value])]
    
find_origin(test, 2705)

In [ ]:
X_test = pd.DataFrame()

for ind in test.data_ind.unique():
    origin = find_origin(test, ind)
    data = test.loc[test.data_ind.isin([ind])]
    #display(data.head())
    data_shift = pd.DataFrame()
    data_shift['DX'] = data['X'] - origin.iloc[0]['X']
    data_shift['DY'] = data['Y'] - origin.iloc[0]['Y']
    data_shift['DZ'] = data['Z'] - origin.iloc[0]['Z']
    data_shift['DTX'] = np.tan(np.arctan(data['TX']) - np.arctan(origin.iloc[0]['TX']))
    data_shift['DTY'] = np.tan(np.arctan(data['TY']) - np.arctan(origin.iloc[0]['TY']))
    X_test = X_test.append(data_shift, ignore_index=True)
    #display(data_shift.head())
    #print(data_shift.shape)

print(X_test.shape)
display(X_test.head())

In [ ]:
Xy_train = pd.DataFrame()

for ind in train.data_ind.unique():
    try:
        origin = find_origin(train, ind)
    except:
        print("Can't find origin {}".format(ind))
        continue
    data = train.loc[train.data_ind.isin([ind])]
    #display(data.head())
    data_shift = pd.DataFrame()
    data_shift['DX'] = data['X'] - origin.iloc[0]['X']
    data_shift['DY'] = data['Y'] - origin.iloc[0]['Y']
    data_shift['DZ'] = data['Z'] - origin.iloc[0]['Z']
    data_shift['DTX'] = np.tan(np.arctan(data['TX']) - np.arctan(origin.iloc[0]['TX']))
    data_shift['DTY'] = np.tan(np.arctan(data['TY']) - np.arctan(origin.iloc[0]['TY']))
    data_shift['signal'] = data['signal']
    Xy_train = Xy_train.append(data_shift, ignore_index=True)
    #display(data_shift.head())
    #print(data_shift.shape)

print(Xy_train.shape)
display(Xy_train.head())

In [ ]:
import lightgbm as lgb

In [19]:
y_train = Xy_train['signal']
X_train = Xy_train.drop(['signal'],axis=1)

In [20]:
lgb_train = lgb.Dataset(X_train, y_train)

In [21]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'max_depth': 15,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 4
}

In [22]:
num_round = 20
lgb.cv(params, lgb_train, num_round, nfold=3)

{'auc-mean': [0.91861424056056418,
  0.92865771403097142,
  0.96562620388190556,
  0.96665617934857229,
  0.97388491142516631,
  0.97342221408200924,
  0.9714089534796434,
  0.97353750321070365,
  0.97444898182159223,
  0.97345174634407927,
  0.97387147130278995,
  0.97401545444330129,
  0.9729677267602358,
  0.97400679129387135,
  0.97444377614472921,
  0.97455360347946318,
  0.97446232508998554,
  0.97519751280964073,
  0.97551230285408819,
  0.97493124780543894],
 'auc-stdv': [0.0025411865881593268,
  0.0014373382456020179,
  0.0007477014417584571,
  0.00096219090787870334,
  0.0010551444940922431,
  0.0010226012627809339,
  0.0010286543667528611,
  0.00087720272813067229,
  0.00080834310048508282,
  0.00082359393066323372,
  0.00075113875693397475,
  0.0006008495309812185,
  0.00057633591748144811,
  0.00057972005097419322,
  0.00058318527166649513,
  0.00063777471864663875,
  0.00065895042064216763,
  0.00060725525067555368,
  0.00055518822515574352,
  0.00053927899918493902]}

In [23]:
num_round = 100
bst = lgb.train(params, lgb_train, num_round)

In [25]:
ypred = bst.predict(X_test)

In [32]:
result = pd.DataFrame(data={'signal': ypred})
result.insert(0, 'id', range(len(result)))

In [33]:
print(result.shape)
result.head()

(13726164, 2)


,id,signal
0,0,0.003818
1,1,0.010382
2,2,0.005745
3,3,0.003707
4,4,0.003450


In [34]:
result.to_csv('data/dm3/result-1.csv',index=False)